# Machine learning в продакшне

Поговорим, от том, как выводить модели в продакшн. Вы уже знакомы с понятием "жизненный цикл ML проекта" - сегодня мы увидим, как этот цикл выглядит в реальной жизни и попробуем вывести в продакшн модель классификации с помощью технологии виртуализации Docker (можно и без docker)

In [12]:
import os

import numpy as np

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  # гарантируем воспроизводимость

run_env = os.getenv('RUN_ENV', 'COLLAB')
if run_env == 'COLLAB':
  from google.colab import drive
  ROOT_DIR = '/content/drive'
  drive.mount(ROOT_DIR)
  print('Google drive connected')
  DRIVE_DATA_DIR = 'ml_course_data'
  root_data_dir = os.path.join(ROOT_DIR, 'MyDrive', DRIVE_DATA_DIR)
else:
  root_data_dir = os.getenv('DATA_DIR', '/srv/data')

if not os.path.exists(root_data_dir):
  raise RuntimeError('Отсутствует директория с данными')
else:
  print('Содержимое директории %s: %s' % (root_data_dir, os.listdir(root_data_dir)[:5]))

Содержимое директории /Users/adzhumurat/PycharmProjects/ai_product_engineer/data: ['client_segmentation.csv', 'messages.db', 'labeled_data_corpus.csv', 'chroma', 'content_description.csv']


# ML в проде: рекомендательные системы

Рекомендательные системы - это обобщённое название для сервисов, которые в ответ на запрос формируют из огромной коллекции доступных сущностей (мы будем называть их "документами", англ. item но в реальной жизни это фильмы в онлайн-кинотеатре или товары в интернет-магазине, а саму такую коллекцию документов для краткости будем называть каталогом) некоторый шорт-лист, который будет наиболее подходящим для данного запроса (такие документы называются "релевантными"). Рекомендательные системы используют для решения своих задач другие алгоритмы ML.
О том, как на основе уже изученных ML-алгоритмов научиться решать задачу поиска релевантных объектов мы поговорим в этом  уроке

## Рекомендательные системы: цели и задачи

Основная задача рекомендательной системы - нарезать шорт-лист из каталога, который будет максимально подходить к контексту, в котором запрашиваются рекомендации.

Под "контекстом" мы понимаем различные параметры мира, который окружает рекомендательную систему:

* с какими документами из каталога уже взаимодействовал пользователь
* какова "сила" взаимодействия - лайк или дизлайк? Если оценка - то какая? Если просмотр фильма - то как долго смотрел?
* в какой стране находится пользователь - в Казахстане смотрят фильм "Рэкетир" а в России "Бумер".
* утро или вечер в момент посмотрения рекомендаций
* что по соцдему - какого пола и возраста пользователь

Задача рекомендаций представляет собой задачу *ранжирования* - мы пытаемся расставить контент в таком порядке, чтобы в начале списка был самый релевантный контент

Вот, например, как выглядит главная страница ivi - по сути, это персонализованная нарезка из доступного каталога контента

![ivi_main](img/ivi_catalog.png)

Различают два вида рекомендаций:

* *content to user*  строятся на основе фидбэка, полученного от пользователя (например, лайкнутые фильмы)
* *content to content* строятся на основе свойств контента найти максимально похожие на него элементы (например, блок "С этим часто покупают" на Ozon)

Рекомендательная система решает задачу персонализации сервиса - это значит, что каждый пользователь  видит свой вариант сервиса - можно сказать, что у вас появляется столько вариантов наполнения страниц вашего сайта, сколько у вас пользователей.

Рекомендательные системы нужны в ситуации, когда пользователь не может сделать выбор из огромного количества возможных вариантов каталога (например, каталога интернет-магазина) и ему нужно как-то помочь, ограничив выбор узким кругом объектов, которые больше всего подходят к текущему контексту - такие объекты и называются *релевантными*. Избавив пользователя от необходимости просматривать огромное число неподходящих объектов, рекомендательная система позволяет быстрее сконвертировать пользовательскую сессию в "целевое действие" - просмотр фильма или покупку гаджета.

Существует два основных метода для поиска релелевантных объектов

* collaborative filtering
* content filtering
* deep learning resys

Модели *коллаборативной фильтрации* используют интуитивное предположение о том, что если у двух пользователей сильно пересекаются два множества контента (с которым они взаимодействовали) то у этих двух пользователей схожие вкусы - на основе этого можно делать рекомендации.

Допустим, у нас есть информация о пользователях `X, Y, Z` и фильмах `A, B, C, D, E`. В таблице символ "+" означает, что пользователь $i$ смотрел фильм $j$:

| - | A | B | C | D | E |
| -- | -- | -- | -- | -- | -- |
| X  | +  | -  | +  | +  | -  |
| Y  | -  | +  | -  | -  | +  |
| Z  | +  | +  | -  | +  | -  |

Как видно, пользователи `X, Z` очень похожи друг на друга, т.к. им обоим нравится фильмы "А" и "D".

В то же время пользователь `Y` не похож на них, он смотрит другие фильмы. Из картинки видно, что если бы нам нужно было бы построить персональные рекомендации для пользователя `X`, то мы бы порекомендовали  ему фильм `B` - такой фильм смотрел пользователь Z и значит, этот фильм понравится пользователю X, потому что данные говорят о том, чт вкусы этих двух пользователей похожи.

*Content-based* фильтрация подбирает похожие элементы на основе свойств контента. Например, в случае фильма его свойства - это актёр, режиссёр, жанр, место действия и т.д. Например, если у нас естьфильм со Стэтхемом, то максимально похожим будет, скорее всего, другой фильм со Стэтхэмом.

Также выделяют *гибридные* рекомендательные системы - в этом случае рекомендации из нескольких моделей (например, `content based` + `collaborative filterig`) используются для формирования итогового ранжирования.

Колаборативные модели обучаются на триплетах (тройках) `(user, item, feedback)`, где для каждой пары `(user, item)` фидбэк может быть двух видов:

* explicit (явный)
* implicit (неявный)

*Явный фидбэк* - это любое явное выражение пользователем своего отношения к контенту: лайк, оценка и т.д. *Неявный фидбэк* - это длительность просмотра фильма, количество заходов на карточку товара, время чтения статьи и т.д.: интуитивно понятно, что если пользователь смотрит фильм долго - то такой фильм, скорее всего, ему нравится, но явного сигнала (например, в виде лайка) мы не получаем.

Задача построения рекомендации решается с помощью классических алгоритмов машинного обучения:

* метод ближайших соседей (knn)
* матричная факторизация
* логистическая регрессия

Рекомендатель отдаёт отранжированный список объектов. На этот список обычно накладываются разнообразные бизнес-правила, позволяющие оптимизировать метрики, важные для бизнеса: например, искуственное повышение новинок в выдаче.

Грамотно построенная рекомендательная система позволяет нарастить продуктовые метрики. Например, в онлайн-кинотеатре:

* растёт конверсия в просмотр т.к. пользователь получает на главной странице самый подходящий контент
* увеличивается длительности смотрения на пользователя, удачно рекомендованный сериал генерирует десятки часов смотрения
* уменьшается отток с сервиса - пользователь за одну сессию может выбрать несколько фильмов, которые будет смотреть несколько заходов подряд.

## KNN рекомендации

Мы попробуем построить простейшую модель коллаборативной фильтрации, пользуясь только информацией о просмотрах контента в онлайн-кинотеатре ivi.

Загрузим исходные данные - там примерно полмиллиона просмотров:

In [13]:
import pandas as pd
import numpy as np

content_views = pd.read_csv(
    os.path.join(root_data_dir, 'content_views.csv.gz'), compression='gzip',
    parse_dates = [3, 4]
)

print('Количество просмотров %s' % content_views.user_id.count())

content_views.head(3)

Количество просмотров 1408597


,user_id,content_id,view_duration,view_ts,dt,platform
0,42,64103,668,2019-04-09 03:34:51+00:00,2019-04-09,Android
1,42,51991,746,2019-04-09 09:08:18+00:00,2019-04-09,Android
2,42,51988,746,2019-04-09 09:46:25+00:00,2019-04-09,Android


Загрузим дополнительную информацию о контенте: жанры, дату появления на сервисе, рейтинг кинопоиска и т.д.

In [14]:
content_description = pd.read_csv(
    os.path.join(root_data_dir, 'content_description.csv.gz'), compression='gzip',
    parse_dates = [2]
)

print('Количество доступного контента %s' % content_description.content_id.count())

content_description.head(3)

Количество доступного контента 141321


,content_id,origin_country,release_date,kinopoisk_rating,compilation_id,genre
0,39261,1.0,2017-10-23,NaN,3519,Боевики
1,62223,56.0,2019-02-18,NaN,4238,Мелодрамы
2,9359,33.0,2011-08-07,NaN,844,Комедийные


Сформируем разреженную матрицу user-item такую, что

* количество строк матрицы совпадает с числом пользователей
* количество столбцов матрицы совпадает с количеством контента
* на пересечении столбца $i$ и строки $j$ стоит единица, если пользователь $i$ смотрел контент $j$, иначе - ноль

Для начала перейдём от индекса контента и индекса пользователя к индексам в разреженной матрице - воспользуемся `LabelEncoder`

In [15]:
from sklearn.preprocessing import LabelEncoder

# кодируем индексы пользователей
user_encoder = LabelEncoder()
user_encoder.fit(content_views.user_id)
# переиндексация контента
content_views = content_views.assign(
    user_index = user_encoder.transform(content_views.user_id)
)
# кодируем индексы контента
item_encoder = LabelEncoder()
item_encoder.fit(content_views.content_id)
# нова переиндексация
content_views = content_views.assign(
    item_index = item_encoder.transform(content_views.content_id)
)

content_views.head()

,user_id,content_id,view_duration,view_ts,dt,platform,user_index,item_index
0,42,64103,668,2019-04-09 03:34:51+00:00,2019-04-09,Android,42,43147
1,42,51991,746,2019-04-09 09:08:18+00:00,2019-04-09,Android,42,35316
2,42,51988,746,2019-04-09 09:46:25+00:00,2019-04-09,Android,42,35313
3,42,4899,1291,2019-04-09 05:39:08+00:00,2019-04-09,Android,42,3844
4,42,33,240,2019-04-09 18:49:45+00:00,2019-04-09,Android,42,33


Теперь у нас есть колонки `user_index, item_index`, которые соответствуют номерам строки и столбца соответственно в матрице user-item. Передадим полученные колонки в конструктор `csr_matrix`, чтобы получить разреженную матрицу

In [16]:
from scipy.sparse import csr_matrix

num_users = content_views.user_index.max() + 1
num_items = content_views.item_index.max() + 1
num_interactions = content_views.shape[0]

user_item = csr_matrix(
    (
        np.ones(num_interactions),
        (
            content_views['user_index'].values,
            content_views['item_index'].values
        )
    ),
    shape=(num_users, num_items)
)
print('sparsity: %.4f' % (num_interactions / (num_users * num_items)))

user_item

sparsity: 0.0003


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1090851 stored elements and shape (124262, 44053)>

Мы видим, что в наше матрице 124262 пользователей и 44053 единиц контента, у матрицы высокая разреженность - менее 1% ненулевых элементов, остальное заполнено нулями.


Разделяем выборку на валидацию и контроль

In [17]:
from sklearn.model_selection import train_test_split

train_ids, test_ids = train_test_split(
    np.arange(start=0, stop=user_item.shape[0], step=1, dtype=np.uint32),
    test_size=0.2
)
print(
    """
        Размер обучающей выборки %d интеракций
        Размер валидационной выборки %d интеракций
    """
    % (train_ids.size, test_ids.size)
)


        Размер обучающей выборки 99409 интеракций
        Размер валидационной выборки 24853 интеракций
    


Чтобы строить рекомендации по колаборативной модели, нам нужен быстрый способ поиска пользователей, у которых схожая история просмотров - наше предположение в том, что похожие пользователи имеют похожую историю просмотров. Для поиска схожих пользователей воспользуемся поиском ближайших соседей по нашей матрице `user-item`:

In [18]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

# обучаемся только на тренировочной части пользователей
model_knn.fit(user_item[train_ids,:])

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

Создадим простейший класс, который умеет искать похожих по истории смотрения пользователей и выдавать рекомендации. Рекомендации - это контент, который смотрели ближашие соседи пользователя, а сам пользователь этот контент не видел

In [19]:
class ColaborativeFilteringKNNRecommender:
    def __init__(self, knn_model, user_item_matrix, num_neighbors):
        self.knn_model = knn_model
        self.user_item_matrix = user_item_matrix
        self.num_neighbors = num_neighbors
        self.top_recs = 50

    def make_recs(self, user_history: csr_matrix, top_recs: int):
        neighbors = model_knn.kneighbors(
            random_user_history,
            self.num_neighbors,
            return_distance=False
        )[0]
        full_recs = user_item[neighbors,:].max(axis=0)
        # рекомендации - это то, что насмотрели ближайшие соседи
        user_history_ids = user_history.nonzero()[1]
        # последовательность id того контента, который смотрели ближайшие соседи
        full_recs_ids = full_recs.nonzero()[1][:self.top_recs]
        # исключаем из рекомендаций то, что уже было у упользователя в историии
        success_recs = np.array([i for i in full_recs_ids if i in user_history_ids])
        print("Число успешных рекомендаций %d из %d" % (success_recs.size, top_recs))

        return np.array([i for i in full_recs_ids if i not in user_history_ids])[:top_recs]


# объект рекомендателя
recommender = ColaborativeFilteringKNNRecommender(
    knn_model=model_knn,
    user_item_matrix=user_item,
    num_neighbors=10
)
print(recommender)

Применяем рекомендатель для случайного пользователя

In [20]:
# пример рекомендаций для случайного пользователя
random_user_index = np.random.choice(test_ids)
random_user_history = user_item.getrow(random_user_index).reshape(1, -1)

recs = recommender.make_recs(random_user_history, top_recs=10)
print('user_index %d, history: %s' % (random_user_index, random_user_history.nonzero()[1][:10]))
print('recommendations: %s' % recs)

Число успешных рекомендаций 1 из 10
user_index 72803, history: [17527 18222 26251 30241 31152 38494]
recommendations: [ 1086  1087  3408  5840  8211  8217  8251 18460 20924 22364]


## Домашняя работа: строим KNN

В реальной жизни KNN-рекомендатель не стоит делать на основе `sklearn.neighbors.NearestNeighbors` - есть готовые реализации, заточенные специально для построения рекомендательных систем. Хорошим примером такой реализации является [пакет implicit](https://benfred.github.io/implicit/api/models/cpu/als.html). В рамках домашней работы предлагается разобраться с реализацией KNN-рекомендателя из этой библиотеки

Мы обучили простейшую рекомендательную систему, основанную на колаборативной фильтрации и поиске ближайших соседей. В следующем уроке мы узнаем, как перейти от рекомендаций основанных на knn к более сложной концепции эмбеддингов

# Эмбеддинги пользователей и контента (скрытые факторы)

Рекомендации, которые строятся на основе KNN, имеют следующий недостаток: фактическим мы запоминаем матрицу `user-item` на этапе тренировки модели - такие модели называются *memory-based*. Это приводит к большому расходу памяти на этапе получения предсказаний, т.к. по каждому пользователю мы вынуждены хранить разреженный вектор большой размерности. При росте количества пользователей модель начинает критически быстро расти по памяти.

Другой подход заключается в применении матричного разложения - когда мы пытаемся приблизить нашу большую разреженную матрицу user-item размерности $m\times n$ двумя плотными матрицами, которые называются матрицей латентных (скрытых) факторов пользователей размерности $m\times f$ и матрицей скрытых факторов контента размерности $n \times f$. О таких методах мы уже говорили в уроке по теме "Снижение размерности" курса "Машинное обучение. Начальный уровень". Такой подход, при котором мы запоминаем не матрицу user-item, а только некий полезный сигнал из этой матрицы, называется *model-based*.

Фокус в том, что $m,n$ - это числа порядка десятков тысяч, а $f$ имеет небольшую размерность (обычно 30-50). Кажому пользователю и каждой единице понтента при таком подходе ставится в соответствие вектор размерности $f$, а произведение вектора пользователя $p_u$ на вектор контента $q_i$ даёт единственное число $r_{ui}$ - меру "релевантности", насколько данный контент понравится данному пользователю

![svd_decomp](img/svd_decomp.png)

$$
r_{ui} = q_i^Tp_u
$$

Для матричного разложения используются различные алгоритмы:

* Singular Vector Decomposition (SVD)
* Alternative Least Squares (ALS)
* Bayesian personalized ranking (BPR)

При решении реальных задач советую начать с BPR - он даёт отличные результаты, но у него есть и недостаток - вычислительная сложность, которая увеличивает время обучения. Эта функция потерь решает задачу ранжирования.

Выбирая между SVD и ALS я рекомендую выбирать ALS, т.к. он более новый и более быстрый.

Мы продемонстрируем работу рекомендательной системы на примере SVD-разложения - для этотго даже не нужно устанавливать дополнительные пакеты, нужная функция есть прямо в пакете `numpy`:

In [21]:
from scipy.sparse.linalg import svds

latent_factors_num = 50
# раскладываем Useк-Item матрицу
user_factors, scale, item_factors = svds(
    user_item.asfptype(),
    k=latent_factors_num,
    return_singular_vectors=True
)
scale = np.diag(np.sqrt(scale))
# эмбеддинги пользователей
user_factors = np.dot(user_factors, scale).astype(np.float32)
# эмбеддинги контента
item_factors = np.dot(scale, item_factors).astype(np.float32)

print(user_item.shape, user_factors.shape, item_factors.T.shape)

(124262, 44053) (124262, 50) (44053, 50)


Спроектируем класс, который будет формировать рекомендации на основе матриц `user_factors` и `item_factors`

In [24]:
class ColaborativeFilteringFactorizationRecommender:
    def __init__(self, user_factors, item_factors, user_item_matrix):
        self.user_factors = user_factors
        self.item_factors = item_factors
        self.user_item_matrix = user_item_matrix

    def make_recs(self, user_index):
        user_factors = self.user_factors[user_index,:]
        user_history_ids = self.user_item_matrix.getrow(user_index).nonzero()[1]
        # рекомендации - это произведение вектора на матрицу
        personal_scores = user_factors.reshape(1, -1).dot(self.item_factors).flatten()
        personal_recs = np.argsort(-personal_scores)[:20]
        return np.array([i for i in personal_recs if i not in user_history_ids])

factorization_recommender = ColaborativeFilteringFactorizationRecommender(
    user_factors=user_factors,
    item_factors=item_factors,
    user_item_matrix=user_item
)

random_user_index = np.random.choice(test_ids)
print('Recommendations:')
factorization_recommender.make_recs(random_user_index)

Recommendations:


array([ 6000,  6003,  5996,  6004,  6005,  5999,  8168,  9900,  8166,
       41375, 19202, 41374,  6576,  8167, 19201, 10873])

Колаборативная модель на основе скрытых факторов пользователя/контента позволяет быстро получить список релевантного контента для каждого пользователя

## Метрики рекомендательных систем

Мы научились делать персональные рекомендации единиц каталога. В этом уроке поговорим о том, как отделить "хорошие" модели от "плохих"

Метрики рекомендательной системы (как и любых других продакшн-систем) можено разделить на три основных группы

* оффлайн-метрики модели
* продуктовые (бизнес) метрики

Продуктовые метрики - это то, как пользователи реагируют на рекомендации. К пользовательским метрикам можно отнести:

* конверсию (в просмотр, в покупку и т.д.) - чем выше конверсия, тем лучше модель
* длительность нахождения на сервисе (чем дольше, тем лучше)
* поизицию, до которой были просмотрены рекомендации - чем меньше позиций пролистали, тем лучше

**Продуктовые метрики** (иногда их называют онлайн-метриками) можно измерить, только проведя АБ-тест, эксперимент на реальных пользователях сервиса. Механика проведения эксперимента следующая мы делим пользователей на две группы: группа А видит старый вариант алгоритма, группа Б - новый вариант. После окончания теста сравнивается значение в тестовой группе и контрольной.

**Оффлайн-метрики** рекомендательных систем позволяют проверить, насколько качественные прогнозы выполняет модель, пользуясь историческими данными.

Для этой задачи можно приспособить известную Вам метрику **RMSE**, которая определяет, насколько точно мы приблизили матрицу *user-item*
$$
\text{RMSE} = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(a_{ij}-\hat{a}_{ij})^2}
$$

Где $a_{ij}$ - истинное значение фидбека пользователя $i$ для контента $j$, а $\hat{a}_{ij}$ - значение фидбэка, которое предсказала наша модель

Это неплохая метрика, например, для задачи регрессии, но для рекомендаций она немного странная - в принципе, нам не важно, насколько хорошо предсказываются оценки 2,3,4 - мы хотим уверенном предсказывать высокие оценки: 8,9,10.

Другой пример метрик - это метрики из задач классификации (precision + recall).

Допустим, у нас есть список рекомендаций из $n=5$ элементов и список фактических просмотров из $k=3$ элементов. В красную рамку обведены те рекомендации, которые были фактически просмотрены - количество таких фильмов $m = 2$

![eval](img/eval.png)

**Recall**  показывает отношение товаров, купленных из рекомендаций к общему числу фактических просмотров

$$
\text{recall} = \frac{m}{k} = \frac{2}{3}
$$

**Precision**  показывает, насколько много из рекомендованного нами попало в итоге в просмотренное

$$
\text{precision} = \frac{m}{n} = \frac{2}{5}
$$

У precision заметен один недостаток - игнорируется порядок, в котором пользователя заинтересовали рекомендованные объекты. Эту задачу решает метрика *average precision at K*. В числителе дисконтируем каждую "единичку", на ту позицию, где она стоит:



$$
\text{ap} = \frac{1}{5}\left( 0 + \frac{1}{2} + \frac{1}{3}\right) = \frac{1}{5}\cdot \frac{5}{6} = \frac{1}{6}
$$

при такой модификации мы заставляем модель не просто угадывать релевантные объекты, но и "поднимать" их выше к началу списка

Кроме перечисленных выше метрик, которые измеряют точность "угадывания", можно мерять и другие показатели

* diversity (разннообразие выдачи) - как сильно в выдаче представлены разные жанры, авторы и т.д.
* coverage (покрытие каталога) - какая часть каталога покрывается рекомендациями (следует избегать случаев, когда всем пользователям рекомендуюется одно и то же подмножество)
* serendipity (индивидуальность) -  берем разницу между вероятностью рекомендации айтема юзеру и всем юзерам (т.е. насколько рекомендация индивидуальна) и суммируем по всем релевантным рекомендованым айтемам. Позволяет определить, насколько выдача "заточена" под пользователя
* novelty (новизна) - как часто попадают в выдачу новые (холодные) элементы

В этом уроке познакомились с метриками. Теперь вы можете отличать хорошие модели рекомендаций от плохих.

В этом модуле мы мознакомились с темой рекомендательных систем. Это область знаний, которую "просто выучить, сложно понять" - потому что с помощью простых алгоритмов ML можно сильно увеличить эффективность вашего бизнеса, но правильное применение этих методов к задаче построения рекомендательной системы - особое искусство, требующее хорошего понимания бизнес-области и знания специфических метрик систем ранжирования

Примерная архитектура полного цикла усредненной ML системы

![full_cycle_ml](./img/recsys_production.jpg)

В этом уроке мы поговорим о том, как обосновать перед бизнесом необходимость внедрения новой ML-системы

### Домашнее задание

В домашке будет задача на реализацию метрик